
# EvasionBench Project Recap: Work, Results, and Decisions

This notebook rebuilds a full project recap from repository evidence.

It answers three questions:
1. What did we build and run?
2. What results did we get?
3. Which decisions did we make, and why?

All values come from files in `artifacts/`, `docs/`, and git history so the notebook stays reproducible.



## Outline

1. Repository + workflow snapshot
2. Phase 3-4 analysis outputs
3. Phase 5-6 modeling outputs
4. Explainability and label diagnostics
5. Phase 7 reporting status
6. Phase 8 optimization and serving decision
7. Decision log with evidence
8. Final recap


In [ ]:

from __future__ import annotations

import csv
import json
import statistics
import subprocess
from datetime import datetime, timezone
from pathlib import Path
from typing import Any


def find_repo_root(start: Path) -> Path:
    for candidate in [start, *start.parents]:
        if (candidate / "AGENTS.md").exists() and (candidate / "artifacts").exists():
            return candidate
    return start


ROOT = find_repo_root(Path.cwd())
print(f"Repo root: {ROOT}")
print(f"Notebook run at (UTC): {datetime.now(timezone.utc).isoformat()}")


def read_json(rel_path: str, default: Any = None) -> Any:
    path = ROOT / rel_path
    if not path.exists():
        print(f"[missing] {rel_path}")
        return {} if default is None else default
    with path.open("r", encoding="utf-8") as f:
        return json.load(f)


def read_csv(rel_path: str) -> list[dict[str, str]]:
    path = ROOT / rel_path
    if not path.exists():
        print(f"[missing] {rel_path}")
        return []
    with path.open("r", encoding="utf-8", newline="") as f:
        return list(csv.DictReader(f))


def fmt(x: Any, digits: int = 4) -> str:
    if isinstance(x, float):
        return f"{x:.{digits}f}"
    return str(x)


def print_table(headers: list[str], rows: list[list[Any]], max_rows: int | None = None) -> None:
    if max_rows is not None:
        rows = rows[:max_rows]
    widths = [len(h) for h in headers]
    for row in rows:
        for i, value in enumerate(row):
            widths[i] = max(widths[i], len(fmt(value)))

    def render_row(values: list[Any]) -> str:
        return " | ".join(fmt(v).ljust(widths[i]) for i, v in enumerate(values))

    print(render_row(headers))
    print("-+-".join("-" * w for w in widths))
    for row in rows:
        print(render_row(row))



## 1) Repository and Workflow Snapshot


In [ ]:

scripts_count = len(list((ROOT / "scripts").glob("*.py")))
src_modules = len(list((ROOT / "src").rglob("*.py")))
test_files = len(list((ROOT / "tests").glob("test_*.py")))

phase_dirs = [
    "artifacts/analysis/phase3",
    "artifacts/analysis/phase4",
    "artifacts/models/phase5",
    "artifacts/models/phase6",
    "artifacts/explainability/phase6",
    "artifacts/diagnostics/phase6",
    "artifacts/reports/phase7",
    "artifacts/models/phase8",
]

print("Script-first implementation footprint")
print(f"- script entrypoints (.py): {scripts_count}")
print(f"- src modules (.py): {src_modules}")
print(f"- test files: {test_files}")
print()

rows = []
for rel in phase_dirs:
    p = ROOT / rel
    file_count = sum(1 for item in p.rglob("*") if item.is_file()) if p.exists() else 0
    rows.append([rel, p.exists(), file_count])

print("Artifact footprint by phase")
print_table(["path", "exists", "file_count"], rows)

print()
print("Recent git timeline (most recent first)")
try:
    log_out = subprocess.check_output(
        ["git", "log", "--pretty=format:%h %ad %s", "--date=short", "-n", "12"],
        cwd=ROOT,
        text=True,
    )
    print(log_out)
except Exception as exc:
    print(f"Could not read git log: {exc}")



## 2) Phase 3-4 Analysis Results


In [ ]:

class_dist = read_json("artifacts/analysis/phase3/core_stats/class_distribution.json", default=[])
length_tests = read_json("artifacts/analysis/phase3/core_stats/length_tests.json", default={})
semantic_h = read_json("artifacts/analysis/phase4/semantic_similarity/hypothesis_summary.json", default={})
question_behavior = read_json("artifacts/analysis/phase4/question_behavior/question_behavior_summary.json", default={})

print("Phase 3: class distribution")
total = sum(item.get("count", 0) for item in class_dist) or 1
rows = []
for item in class_dist:
    label = item.get("label", "unknown")
    count = item.get("count", 0)
    pct = 100.0 * count / total
    rows.append([label, count, f"{pct:.2f}%"])
print_table(["label", "count", "share"], rows)

print()
print("Phase 3: length hypothesis tests")
for metric_name in ["answer_length", "question_length"]:
    block = length_tests.get(metric_name, {})
    kruskal = block.get("kruskal", {})
    p_value = kruskal.get("p_value")
    interpretation = block.get("interpretation", "")
    print(f"- {metric_name}: p_value={fmt(p_value, 6)} | {interpretation}")

print()
print("Phase 4: semantic hypothesis summary")
for item in semantic_h.get("hypotheses", []):
    print(f"- {item.get('id')}: {item.get('name')} -> {item.get('finding')}")

print()
print("Phase 4: question behavior refusal spread (top 5)")
spreads = question_behavior.get("refusal_spread_by_question_type", [])
spreads_sorted = sorted(spreads, key=lambda x: x.get("refusal_rate_spread", 0), reverse=True)
rows = []
for item in spreads_sorted[:5]:
    rows.append([
        item.get("question_type"),
        item.get("min_refusal_rate"),
        item.get("max_refusal_rate"),
        item.get("refusal_rate_spread"),
    ])
print_table(["question_type", "min_refusal", "max_refusal", "spread"], rows)



## 3) Phase 5-6 Modeling Results


In [ ]:

phase5_summary = read_json("artifacts/models/phase5/run_summary.json", default={})
phase5_metrics_files = {
    "logreg": read_json("artifacts/models/phase5/logreg/metrics.json", default={}),
    "tree": read_json("artifacts/models/phase5/tree/metrics.json", default={}),
    "boosting": read_json("artifacts/models/phase5/boosting/metrics.json", default={}),
    "tree_boosting": read_json("artifacts/models/phase5/tree_boosting/metrics.json", default={}),
}
phase6_transformer = read_json("artifacts/models/phase6/transformer/metrics.json", default={})

print("Phase 5 canonical summary (run_summary.json)")
rows = []
for family, metrics in sorted(phase5_summary.items()):
    rows.append([
        family,
        metrics.get("accuracy"),
        metrics.get("f1_macro"),
        metrics.get("precision_macro"),
        metrics.get("recall_macro"),
    ])
print_table(["family", "accuracy", "f1_macro", "precision_macro", "recall_macro"], rows)

if rows:
    best = max(rows, key=lambda r: r[2] if isinstance(r[2], float) else -1)
    print()
    print(f"Best classical family by macro-F1 (from summary): {best[0]} ({fmt(best[2])})")

print()
print("Phase 5 direct metrics files (for sanity check)")
rows = []
for family, metrics in phase5_metrics_files.items():
    rows.append([
        family,
        metrics.get("accuracy"),
        metrics.get("f1_macro"),
        metrics.get("precision_macro"),
        metrics.get("recall_macro"),
    ])
print_table(["family", "accuracy", "f1_macro", "precision_macro", "recall_macro"], rows)

print()
print("Phase 6 transformer metrics")
print(phase6_transformer)



## 4) Explainability and Label Quality


In [ ]:

xai_summary = read_json("artifacts/explainability/phase6/xai_summary.json", default={})
label_diag = read_json("artifacts/diagnostics/phase6/label_diagnostics_summary.json", default={})
near_dupes = read_csv("artifacts/diagnostics/phase6/near_duplicate_pairs.csv")
suspects = read_csv("artifacts/diagnostics/phase6/suspect_examples.csv")
outliers = read_csv("artifacts/diagnostics/phase6/outlier_examples.csv")

print("Explainability families and explainers")
rows = []
for family, payload in sorted(xai_summary.items()):
    rows.append([
        family,
        payload.get("explainer_type"),
        payload.get("n_features"),
        payload.get("n_samples"),
    ])
print_table(["family", "explainer", "n_features", "n_samples"], rows)

print()
print("Label diagnostics summary")
for k in [
    "training_size",
    "quality_score",
    "label_issues",
    "near_duplicate_issues",
    "outlier_issues",
    "random_state",
    "git_sha",
]:
    print(f"- {k}: {label_diag.get(k)}")

print()
print("Raw diagnostics file row counts")
print(f"- near_duplicate_pairs.csv rows: {len(near_dupes)}")
print(f"- suspect_examples.csv rows: {len(suspects)}")
print(f"- outlier_examples.csv rows: {len(outliers)}")



## 5) Phase 7 Reporting Pipeline Status


In [ ]:

report_run = read_json("artifacts/reports/phase7/run_summary.json", default={})
manifest = read_json("artifacts/reports/phase7/provenance_manifest.json", default={})
traceability = read_json("artifacts/reports/phase7/report_traceability.json", default={})

print("Phase 7 run summary")
for k in ["pipeline", "status", "started_at", "completed_at", "git_sha"]:
    print(f"- {k}: {report_run.get(k)}")

failure = report_run.get("failure") or {}
if failure:
    print()
    print("Failure detail (if any)")
    for k in ["stage", "status", "exit_code", "hint", "log"]:
        print(f"- {k}: {failure.get(k)}")

sections = manifest.get("sections", {})
analysis_count = len(sections.get("analyses", []))
model_count = len(sections.get("models", []))
explainability_count = len(sections.get("explainability", []))
diagnostics_count = len(sections.get("diagnostics", []))

print()
print("Manifest coverage")
print(f"- analyses tracked: {analysis_count}")
print(f"- models tracked: {model_count}")
print(f"- explainability tracked: {explainability_count}")
print(f"- diagnostics tracked: {diagnostics_count}")

trace_items = traceability.get("items", {})
print(f"- traceability item count: {len(trace_items)}")



## 6) Phase 8 Optimization and Serving Decision


In [ ]:

phase8_selected = sorted((ROOT / "artifacts" / "models").glob("phase8*/selected_model.json"))
rows = []
for path in phase8_selected:
    rel = path.relative_to(ROOT).as_posix()
    payload = read_json(rel, default={})
    metrics = payload.get("metrics", {})
    rows.append([
        path.parent.name,
        payload.get("best_model_family"),
        payload.get("winner_trial_id"),
        payload.get("winner_rule"),
        payload.get("accuracy_floor"),
        metrics.get("accuracy"),
        metrics.get("f1_macro"),
        payload.get("random_state"),
    ])

print("Optimization runs with selected_model.json")
print_table(
    [
        "run",
        "family",
        "winner_trial_id",
        "winner_rule",
        "acc_floor",
        "accuracy",
        "f1_macro",
        "seed",
    ],
    rows,
)

canonical = read_json("artifacts/models/phase8/selected_model.json", default={})
canon_metrics = canonical.get("metrics", {})
print()
print("Canonical serving selection (artifacts/models/phase8/selected_model.json)")
print(f"- best_model_family: {canonical.get('best_model_family')}")
print(f"- winner_trial_id: {canonical.get('winner_trial_id')}")
print(f"- winner_rule: {canonical.get('winner_rule')}")
print(f"- accuracy_floor: {canonical.get('accuracy_floor')}")
print(f"- accuracy: {canon_metrics.get('accuracy')}")
print(f"- f1_macro: {canon_metrics.get('f1_macro')}")

seed_rows = [r for r in rows if r[0].startswith("phase8_seed") and isinstance(r[6], float)]
if seed_rows:
    f1_values = [r[6] for r in seed_rows]
    print()
    print(f"Seed-run macro-F1 mean: {statistics.mean(f1_values):.4f}")
    print(f"Seed-run macro-F1 stdev: {statistics.pstdev(f1_values):.4f}")



## 7) Decision Log (What We Chose and Why)


In [ ]:

case_study_text = (ROOT / "docs" / "case_study.md").read_text(encoding="utf-8") if (ROOT / "docs" / "case_study.md").exists() else ""
brief_text = (ROOT / "docs" / "hiring_manager_brief.md").read_text(encoding="utf-8") if (ROOT / "docs" / "hiring_manager_brief.md").exists() else ""
opt_text = (ROOT / "docs" / "model_optimization_guide.md").read_text(encoding="utf-8") if (ROOT / "docs" / "model_optimization_guide.md").exists() else ""

canonical = read_json("artifacts/models/phase8/selected_model.json", default={})
canon_metrics = canonical.get("metrics", {})

decisions = [
    (
        "Script-first workflow",
        "All executable logic moved to scripts/src and phase outputs are contract-tested.",
        "Evidence: AGENTS.md + docs/analysis_workflow.md",
    ),
    (
        "Use question [SEP] answer representation",
        "Feature strategy ties question-answer context into one text stream for TF-IDF pipelines.",
        "Evidence: AGENTS.md and docs/architecture.md",
    ),
    (
        "Classical baseline emphasis first",
        "Interpretable, fast baselines established stable benchmark behavior before advanced modeling.",
        "Evidence: docs/case_study.md + docs/hiring_manager_brief.md",
    ),
    (
        "Optimize with macro-F1 + accuracy floor",
        "Selection policy protects minority-class quality while preventing unacceptable accuracy regression.",
        f"Evidence: docs/model_optimization_guide.md + phase8 selected_model.json (floor={canonical.get('accuracy_floor')}, f1={canon_metrics.get('f1_macro')})",
    ),
    (
        "Serve via canonical selected_model.json",
        "Runtime resolution uses a deterministic selected-model file for reproducible deployment.",
        "Evidence: docs/model_optimization_guide.md serving default section",
    ),
]

print("Decisions and rationale")
for i, (decision, rationale, evidence) in enumerate(decisions, start=1):
    print(f"{i}. {decision}")
    print(f"   - rationale: {rationale}")
    print(f"   - {evidence}")

print()
print("Cross-check: exact phrase presence in docs")
checks = [
    ("case_study_has_logreg_why", "Why Logistic Regression won" in case_study_text),
    ("brief_has_tradeoffs", "Engineering Decisions and Tradeoffs" in brief_text),
    ("opt_has_accuracy_floor", "Accuracy floor" in opt_text),
]
for key, passed in checks:
    print(f"- {key}: {passed}")



## 8) Final Recap


In [ ]:

phase5_summary = read_json("artifacts/models/phase5/run_summary.json", default={})
canonical = read_json("artifacts/models/phase8/selected_model.json", default={})
report_run = read_json("artifacts/reports/phase7/run_summary.json", default={})
label_diag = read_json("artifacts/diagnostics/phase6/label_diagnostics_summary.json", default={})

phase5_best = None
if phase5_summary:
    phase5_best = max(
        phase5_summary.items(),
        key=lambda kv: kv[1].get("f1_macro", float("-inf")),
    )[0]

recap = {
    "project": "EvasionBench",
    "workflow": "script-first pipeline with artifact contracts",
    "phase5_best_classical_family": phase5_best,
    "phase8_canonical_family": canonical.get("best_model_family"),
    "phase8_canonical_trial": canonical.get("winner_trial_id"),
    "phase8_canonical_accuracy": canonical.get("metrics", {}).get("accuracy"),
    "phase8_canonical_f1_macro": canonical.get("metrics", {}).get("f1_macro"),
    "label_quality_score": label_diag.get("quality_score"),
    "phase7_report_status": report_run.get("status"),
}

print("Project recap summary")
for k, v in recap.items():
    print(f"- {k}: {v}")

print()
print("Notebook complete: this run reconstructed work, outcomes, and decisions from repository evidence.")
